In [23]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [24]:
import glob
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import imageio
from sklearn.feature_extraction.image import extract_patches_2d

RND_SEED = 7777

In [25]:
def is_defective(mask_patch, th=0.01):
    """
    Check if @mask_patch is defective i.e. the percentage of white pixel is > @th
        - if the mask associated to the extracted patch is defective -> the patch is saved with a '*KO.png' extension
        - if the mask associated to the extracted patch is NOT defective -> the patch is saved with a '*OK.png' extension
    """
    return np.sum(mask_patch) > 255 * mask_patch.shape[0] * mask_patch.shape[1] * th


In [26]:
def create_patches(img_path, n, size, mask_path=None):
    """
    Load imgs and masks and extract @n random patches possibly with overlap. 
    If @mask_path is not set, only patch from input images are extracted
    """
    # load imgs
    input_img_list = glob.glob(os.path.join(img_path, '*'))
    print('Detected %d images from folder --> %s' % (len(input_img_list), os.path.abspath(img_path)))
    imgs = np.array([imageio.imread(f) for f in input_img_list])
    # extract patches
    imgs_patches = np.array([extract_patches_2d(img, (size, size), n, RND_SEED) for img in imgs], dtype=np.uint8)
    # create folder to store patches
    imgs_patches_path = Path(img_path).parents[0].joinpath(os.path.basename(img_path) + '_patch')
    imgs_patches_path.mkdir(exist_ok=True)
    # save to folder
    if not(mask_path):
        print('No mask path has been specified...')
        [misc.imsave(os.path.join(imgs_patches_path, os.path.basename(im_path).split('.')[-2] + '_PATCH_' + str(n) + '.png'), 
                                imgs_patches[id,n,:,:]) for id,im_path in zip(range(imgs_patches.shape[0]), input_img_list) for n in range(imgs_patches.shape[1])]
    else: 
        # load masks
        input_mask_list = glob.glob(os.path.join(mask_path, '*'))
        print('Detected %d masks from folder --> %s' % (len(input_mask_list), os.path.abspath(mask_path)))
        masks = np.array([imageio.imread(f) for f in input_mask_list])
        # extract patches
        masks_patches = np.array([extract_patches_2d(mask, (size, size), n, RND_SEED) for mask in masks], dtype=np.uint8)
        # create folder to store patches
        masks_patch_path = Path(mask_path).parents[0].joinpath(os.path.basename(mask_path) + '_patch')
        masks_patch_path.mkdir(exist_ok=True)
        for (id, m_path, im_path) in zip(range(masks_patches.shape[0]), input_mask_list, input_img_list):
            for n in range(masks_patches.shape[1]):
                curr_img_patch = imgs_patches[id, n, :, :]
                curr_mask_patch = masks_patches[id, n, :, :]
                if is_defective(curr_mask_patch):
                    imageio.imsave(os.path.join(imgs_patches_path, os.path.basename(im_path).split('.')[-2] + '_PATCH_' + str(n) + '_KO.png'), 
                                curr_img_patch)
                    imageio.imsave(os.path.join(masks_patch_path, os.path.basename(m_path).split('.')[-2] + '_PATCH_' + str(n) + '_KO.png'), 
                                curr_mask_patch)
                else:
                    imageio.imsave(os.path.join(imgs_patches_path, os.path.basename(im_path).split('.')[-2] + '_PATCH_' + str(n) + '_OK.png'), 
                                curr_img_patch)
                    imageio.imsave(os.path.join(masks_patch_path, os.path.basename(m_path).split('.')[-2] + '_PATCH_' + str(n) + '_OK.png'), 
                                curr_mask_patch)

In [27]:
img_path = "data/images"
mask_path = "data/masks"
n= 5
size = 100

create_patches(img_path, n, size, mask_path) 

Detected 28 images from folder --> C:\Users\User\0.Python\Patches\data\images
Detected 28 masks from folder --> C:\Users\User\0.Python\Patches\data\masks
